In [ ]:
# importando librerias
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

2. Keypoints using MP Holistic


In [121]:
mp_holistic = mp.solutions.holistic #Modelo Holistico
mp_drawing = mp.solutions.drawing_utils # Draw utilities dibuja puntos en la cada

In [122]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


In [123]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

Este funciona

In [124]:
# Inicializar MediaPipe Drawing y Holistic
mp_holistic = mp.solutions.holistic  # Modelo Holístico
mp_drawing = mp.solutions.drawing_utils  # Herramientas para dibujar puntos

def mediapipe_detection(image, model):
    # Convertir la imagen de BGR a RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # La imagen ya no es modificable
    image.flags.writeable = False

    # Realizar la predicción
    results = model.process(image)

    # Volver a hacer la imagen modificable
    image.flags.writeable = True

    # Convertir la imagen de vuelta a BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image, results


def draw_landmarks(image, results):
    # Dibujar los landmarks de la cara
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

    # Dibujar los landmarks del cuerpo (pose)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    # Dibujar los landmarks de la mano izquierda
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # Dibujar los landmarks de la mano derecha
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    return image


##Funciona

In [125]:
def draw_styled_landmarks(image, results):
    # Draw face connections (use FACEMESH_TESSELATION for face)
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION,  # Updated connection attribute
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )

    # Draw pose connections
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

    # Draw left hand connections
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.left_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

    # Draw right hand connections
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

    return image, results


In [126]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

4. Setup Folders for Collection


In [127]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Delete_Me')  # Directory to save the data

# Actions that we try to detect
actions = np.array(['ayer', 'hoy',  'ayuda'])

# Thirty videos worth of data
no_sequences = 1

# Videos are going to be 30 frames in length
sequence_length = 10

In [128]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

5. Collect Keypoint Values for Training and Testing

In [129]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    text = 'COMIENZA'
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 3  
                    thickness = 6
                    color = (0, 255, 0)
                    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
                    center_x = image.shape[1] // 2 - text_width // 2
                    center_y = image.shape[0] // 2 + text_height // 2
                    cv2.putText(image, text, (center_x, center_y), font, font_scale, color, thickness, cv2.LINE_AA)

                    recolecta_text = f'Recolectando datos {action} fotograma numero {sequence}'
                    font_scale2 = 1.2  
                    thickness2 = 3
                    color2 = (255, 0, 0)
                    cv2.putText(image, recolecta_text, (30, 50), font, font_scale2, color2, thickness2, cv2.LINE_AA)

                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    recolecta_text = f'Recolectando datos {action} fotograma numero {sequence}'
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale2 = 1.2 
                    thickness2 = 3
                    color2 = (255, 0, 0)
                    cv2.putText(image, recolecta_text, (30, 50), font, font_scale2, color2, thickness2, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()


In [130]:
import glob

# Ruta a los archivos npy de una acción y secuencia específica
npy_files = glob.glob(os.path.join(DATA_PATH, action, str(sequence), '*.npy'))

for npy_file in npy_files:
    data = np.load(npy_file)
    print(f'Archivo: {npy_file}')
    print(data)
    print('-' * 40)

Archivo: Delete_Me\ayuda\0\0.npy
[ 0.46536538  0.19302914 -0.54687083 ...  0.14901023  0.17872655
 -0.01881366]
----------------------------------------
Archivo: Delete_Me\ayuda\0\1.npy
[ 0.46539766  0.1930075  -0.53117478 ...  0.14960346  0.1786826
 -0.0160711 ]
----------------------------------------
Archivo: Delete_Me\ayuda\0\2.npy
[ 0.46532103  0.19296467 -0.54774815 ...  0.14679283  0.17965735
 -0.02099228]
----------------------------------------
Archivo: Delete_Me\ayuda\0\3.npy
[ 0.46532115  0.19296113 -0.55423808 ...  0.14591013  0.18110046
 -0.0217589 ]
----------------------------------------
Archivo: Delete_Me\ayuda\0\4.npy
[ 0.46534073  0.19311078 -0.55637306 ...  0.14582771  0.18035781
 -0.02194662]
----------------------------------------
Archivo: Delete_Me\ayuda\0\5.npy
[ 0.4653469   0.192792   -0.56508821 ...  0.14539345  0.18119252
 -0.02270865]
----------------------------------------
Archivo: Delete_Me\ayuda\0\6.npy
[ 0.46535152  0.19271968 -0.57509178 ...  0.145915